## Web Crawling

- requests
- beautifulsoup
- selenium

In [ ]:
!pip install BeautifulSoup4
!pip install selenium

### S&P 500

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_sp500():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    doc = BeautifulSoup(response.text, 'html.parser')
    symbol = pd.Series([data.text.replace('\n','') for data in doc.select('#constituents > tbody > tr > td:nth-child(1)')], name='Symbol')
    security = pd.Series([data.text.replace('\n','') for data in doc.select('#constituents > tbody > tr > td:nth-child(2)')], name='Security')
    gics_sector = pd.Series([data.text.replace('\n','') for data in doc.select('#constituents > tbody > tr > td:nth-child(4)')], name='GICS')
    ticker = pd.Series([data.text.replace('\n','') for data in doc.select('#constituents > tbody > tr > td:nth-child(8)')], name='CIK')
    sp500_data = pd.concat([symbol, security, gics_sector, ticker], axis=1)
    return sp500_data

In [ ]:
get_sp500()

### ETF

In [ ]:
!pip install -U fake-useragent

In [ ]:
from fake_useragent import UserAgent

- TIGER 200 IT (139260)
- TIGER 헬스케어 (143860)

In [ ]:
contents = []
for i in range(1, 11):
    url = 'https://finance.naver.com/item/sise_day.nhn?code=139260&page={}'.format(i)
    headers = { 'user-agent' :  UserAgent().chrome}
    response = requests.get(url, headers=headers)
    doc = BeautifulSoup(response.text, 'html5lib')  # 속도가 느림 / 정확
    headers = [h.text for h in doc.select('body > table.type2 > tbody > tr:nth-child(1) > th')]
    #3번째부터 7번째까지 / 11번째부터 15번째까지
    for i in range(3, 8):
        row = doc.select('body > table.type2 > tbody > tr:nth-child({}) > td '.format(i))
        contents.append([data.text.replace('\n', '').replace('\t','') for data in row])
    for i in range(11, 16):
        row = doc.select('body > table.type2 > tbody > tr:nth-child({}) > td '.format(i))
        contents.append([data.text.replace('\n', '').replace('\t','') for data in row])

In [ ]:
etf_data = pd.DataFrame(contents, columns=headers)
etf_data.set_index('날짜', inplace=True)
etf_data.applymap(lambda x : int(x.replace(',','')))

### selenium (뉴스)

In [ ]:
from selenium import webdriver

In [ ]:
pg = 3
url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=101#&date=%2000:00:00&page={}'.format(pg)
driver = webdriver.Chrome()
driver.get(url)

In [ ]:
news = driver.find_elements_by_css_selector('#section_body li')
len(news)

In [ ]:
news[0].find_element_by_css_selector('#section_body li > dl > dt:nth-child(2) > a').text

In [ ]:
news_list = []

for article in news:
    try: 
        title = article.find_element_by_css_selector('#section_body li > dl > dt:nth-child(2) > a').text
        news_list.append(title)
    except:
        continue 
news_list

In [ ]:
len(news_list)

In [ ]:
#드라이버 종료!
driver.quit()